<a href="https://colab.research.google.com/github/Mohamed28112003/Muffakir/blob/main/Notebooks/DATA_GENERATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers
!pip install -U bitsandbytes
!pip install langchain_groq
!pip install langchain
!pip install -U langchain-community
!pip install tiktoken


In [109]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import pandas as pd
from langchain.document_loaders import TextLoader
from langchain_groq import ChatGroq
import time
from langchain.schema import HumanMessage
from google.colab import files
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from typing import Tuple, List, Dict, Optional, Any
from langchain.schema import Document
import re

In [110]:
class TextProcessor:

    @staticmethod
    def split_text_recursive(documents: List[Document], chunk_size: int, chunk_overlap: int):
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap
        )
        return text_splitter.split_documents(documents)

    @staticmethod
    def clean_arabic_text(text):
        text = re.sub(r'Page number: \d+', '', text)
        text = re.sub(r'- \d+ -', '', text)
        text = re.sub(r'[^؀-ۿ0-9\s]', '', text)
        text = re.sub(r'[^\w\s؀-ۿ0-9]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    @staticmethod
    def clean_metadata(metadata):
        if 'source' in metadata:
            source_path = metadata['source']
            filename = source_path.split('/')[-1]
            filename_without_extension = re.sub(r'\.txt$', '', filename)
            cleaned_metadata = re.sub(r'_', ' ', filename_without_extension)
            return cleaned_metadata
        return None

    @staticmethod
    def process_documents(documents: List[Document], chunk_size: int, chunk_overlap: int):
        for doc in documents:
            doc.page_content = TextProcessor.clean_arabic_text(doc.page_content)
            if isinstance(doc.metadata, dict):
                doc.metadata['source'] = TextProcessor.clean_metadata(doc.metadata)

        doc_splits = TextProcessor.split_text_recursive(documents, chunk_size, chunk_overlap)

        for i, doc in enumerate(doc_splits):
            doc.metadata['chunk_id'] = i + 1

        return doc_splits


Your final data frame

In [111]:
final_df = pd.DataFrame()


In [112]:
loader = TextLoader("/content/المالية_العامة_الجزء_الثاني_د_رشدي_الموزانة_العامة.txt")
data = loader.load()

In [113]:
text_processor = TextProcessor()
processed_docs = text_processor.process_documents(data, chunk_size=600, chunk_overlap=100)
processed_docs[0].page_content







'رقم الصفحه 1 مبادئ المالية العامة الجزء الثاني الموازنة العامة د رشدي إبراهيم السيد أبو كريمه قسم الاقتصاد والمالية العامة كلية الحقوق، جامعة عين شمس ٥١٤٤٦ ٢٠٢٤ م رقم الصفحه 2 الباب الثالث الموازنة العامة 2 رقم الصفحه 3 3 رقم الصفحه 4 مقدمة تلعب الموازنة العامة دورًا محوريًا فى الدراسات الاقتصادية والمالية والاجتماعية والسياسية، حيث تؤدى إلى تحقيق التوازن فى توزيع الدخل القومى بما يحقق العدالة والتوازن الاجتماعى فى المجتمع، وهو ما يواكب نمو الحكومات الديمقراطية اقتصاديًا فتستخدم الموازنة العامة كأداة فعالة فى تحقيق التنمية الاقتصادية وتشجيع الاستثمار، فهى تهدف إلى بحث السبل التى من خلالها أن تؤدى إلى'

In [114]:

#  you can optimize this prompt if you need but keep it with the same format please :)

prompt = """

Based on the following text, generate one Question and its corresponding Answer with detailes.
For each question you formulate, provide a comprehensive answer with details immediately following the question,
separated by a vertical bar "|" provide only 1 questions.
If the chunk lacks useful information (e.g., random data, incomplete sentences, or irrelevant content), return none
The context to analyze is as follows:  {context} .
if there
For Example :
## Context:
البعيد أو الغائي لقانون الإجراءات الجنائية يمثل ذات الهدف بالنسبة لكافة العلوم الجنائية.
رابعاً - وظيفة قانون الإجراءات الجنائية:
يمكن القول بأن الوظيفة الأساسية لهذا القانون هي تنظيم العدالة الجنائية؛ وتحقيق هذه
الغاية يتطلب الموائمة بين اعتبارين. يتمثل الاعتبار الأول في وضع آلية يُمكن من خلالها
تفعيل نصوص قانون العقوبات حماية للمصالح الاجتماعية العامة المتعددة، ويتجلى الاعتبار
الثاني في ضمان حقوق الفرد وحريته في مواجهة السلطات الإجرائية الجنائية.

# Your Response Sholud Be Like This format only do not add any thing else
Question : وظيفة قانون الإجراءات الجنائية | يمكن القول بأن الوظيفة الأساسية لهذا القانون هي تنظيم العدالة الجنائية؛ وتحقيق هذه
الغاية يتطلب الموائمة بين اعتبارين. يتمثل الاعتبار الأول في وضع آلية يُمكن من خلالها
تفعيل نصوص قانون العقوبات حماية للمصالح الاجتماعية العامة المتعددة، ويتجلى الاعتبار
الثاني في ضمان حقوق الفرد وحريته في مواجهة السلطات الإجرائية الجنائية.



"""

In [115]:
list_of_api = [
    "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",
    "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",
    "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",
    " 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",
    "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",
    "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",
    "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",
    "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",
     "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",
     "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",
     "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",
    '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',
    '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',
    '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',
    "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",
    "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",
    "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"
]

# ADD YOUR APIs IN THE HEAD OF THE LIST

In [116]:
def initialize_llm(api_key):
    return ChatGroq(
        api_key=api_key,
        model="llama-3.3-70b-versatile",
        temperature=0.7,
        #max_tokens=1024
    )

In [117]:
# Initialize the ChatGroq model with the first API key
api_key_index = 0

llm = initialize_llm(list_of_api[api_key_index])

In [ ]:
results = []
i = 0

for doc in processed_docs:
    Final_prompt = prompt.format(context=doc.page_content)
    messages = [HumanMessage(content=Final_prompt)]

    while True:
        try:
            response = llm.invoke(messages)
            break

        except Exception as e:
            error_message = str(e)
            print(f"Encountered an error: {error_message}")

            # Check if it's a rate limit error or service unavailable error
            if 'rate limit' in error_message.lower() or '429' in error_message:
                print(f"################################## Rate limit reached for API key ################################## {list_of_api[api_key_index]}. Switching API key...")
                api_key_index = (api_key_index + 1) % len(list_of_api)
                llm = initialize_llm(list_of_api[api_key_index])
                time.sleep(2)  # Wait before retrying
            elif '503' in error_message or 'Service Unavailable' in error_message:
                print("Service is unavailable, retrying after a delay...")
                time.sleep(5)  # Longer wait time for service errors
            else:
                # Handle unexpected errors by switching API key
                print(f"Unexpected error. Switching API key: {list_of_api[api_key_index]}")
                api_key_index = (api_key_index + 1) % len(list_of_api)
                llm = initialize_llm(list_of_api[api_key_index])
                time.sleep(2)  # Small delay before retrying

    # Split the response into individual question-answer pairs
    qa_pairs = response.content.split("Question :")
    print("Counter:", i)
    i += 1

    for qa in qa_pairs[1:]:  # Skip the first empty part
        question_answer = qa.split(" | ")
        if len(question_answer) == 2:
            question = question_answer[0].strip()
            answer = question_answer[1].strip()
            print("Question:", question)
            print("Answer:", answer)

            results.append({
                "context": doc.page_content,
                "question": question,
                "answer": answer
            })



Counter: 0
Question: دور الموازنة العامة فى المجتمع
Answer: تلعب الموازنة العامة دورًا محوريًا فى الدراسات الاقتصادية والمالية والاجتماعية والسياسية، حيث تؤدى إلى تحقيق التوازن فى توزيع الدخل القومى بما يحقق العدالة والتوازن الاجتماعى فى المجتمع، وهو ما يواكب نمو الحكومات الديمقراطية اقتصاديًا فتستخدم الموازنة العامة كأداة فعالة فى تحقيق التنمية الاقتصادية وتشجيع الاستثمار، فهى تهدف إلى بحث السبل التى من خلالها أن تؤدى إلى تحقيق هذه الأهداف.
Counter: 1
Question: ما هو الغرض من تحقيق التنمية الاقتصادية وتشجيع الاستثمار في الموازنة العامة للدولة
Answer: يهدف إلى بحث السبل التى من خلالها أن تؤدى إلى توفير الأموال لتسيير المرافق العامة بإنتظام وأطراد، مع ضرورة الاستخدام الأمثل لهذه الأموال بأقصى كفاءة اقتصادية ممكنة.
Counter: 2
Question: ما هو مفهوم الموازنة العامة للدولة
Answer: تُعرف الموازنة العامة للدولة بأنها بيان تقديرى عن فترة مالية مستقبلية غالبًا ما تكون سنة، توضح الإيرادات .
Counter: 3
Question: ما هي الموازنة العامة للدولة
Answer: هي بيان تقديرى عن فترة مالية مستقبلية غالبًا ما 

In [82]:

# Convert results to a DataFrame
df = pd.DataFrame(results)


In [83]:
df.head()

,context,question,answer
0,رقم الصفحه 1 مبادئ المالية العامة الجزء الثاني...,دور الموازنة العامة فى تحقيق التوازن الاجتماعى,تلعب الموازنة العامة دورًا محوريًا فى الدراسات...
1,تحقيق التنمية الاقتصادية وتشجيع الاستثمار، فهى...,ما هي أهمية الموازنة العامة للدولة,الموازنة العامة للدولة تهدف إلى بحث السبل التى...
2,الخامس أثر تحرير سعر الصرف على تحقيق الاستقرار...,ما هو مفهوم الموازنة العامة للدولة,تُعرف الموازنة العامة للدولة بأنها بيان تقديرى...
3,الموازنة العامة للدولة بأنها بيان تقديرى عن فت...,ما هي الموازنة العامة للدولة,هي بيان تقديرى عن فترة مالية مستقبلية غالبًا م...
4,عامة خلال فترة زمنية مستقبلية غالبًا ما تقدر ب...,من تقوم بإعداد الموازنة العامة للدولة,يتم إعداد الموازنة العامة التقديرية والتنبؤات ...


In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:


print("Context:\n", df.loc[2, "context"])

print("Question:\n", df.loc[2, "question"])
print("Answer:\n", df.loc[2, "answer"])


In [ ]:

#pd.set_option('display.max_colwidth', None)


In [ ]:
df.tail(10)


In [ ]:
# CONCAT THE DF WITH TH FINAL_DF

final_df = pd.concat([final_df, df], ignore_index=True)



In [ ]:
final_df.shape

# Run this cell after you finish all the pages of the book

In [ ]:
# replace the book name


final_df.to_csv("الوسيط_في_تنازع_القوانين_د_محمود_لطفي.csv", index=False)

files.download("الوسيط_في_تنازع_القوانين_د_محمود_لطفي.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>